# Cerebus llama3.1 70B PricePredictor

## Prompt:

Create a class that Trains a model given stock prices and indicators (rsi, on balance volume, detrended price oscilator), and produces target predictions of for the next 3 days open, low, and close. The model should also contain methods that will perform Baysian optimization of the LSTM's model hyper parameters. The class should use yfinance to pull daily stock price data. The class should also use mplfinance to plot the price data along with the rescaled open, low and close, predictions.

### Correction...

Please re-write the code using keras 3.5 which nolonger has keras.wrappers, and make use of scikeras.wrappers instead.


In [15]:
%matplotlib notebook

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from scipy import signal
from scipy.signal import find_peaks
from scipy.stats import kurtosis, skew
from sklearn.model_selection import RandomizedSearchCV
from scikeras.wrappers import KerasRegressor
from scipy.stats import randint
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from bayes_opt import BayesianOptimization

class StockPredictor:
    def __init__(self, ticker, start_date, end_date):
        self.ticker = ticker
        self.start_date = start_date
        self.end_date = end_date
        self.data = self.get_stock_data()
        self.orig_data = self.data.copy()
        self.scaler = None

    def get_stock_data(self):
        data = yf.download(self.ticker, start=self.start_date, end=self.end_date)
        # Move date from index to column
        data.reset_index(inplace=True)
        data.rename(columns={'index': 'Date'}, inplace=True)
        return data

    def calculate_indicators(self):
        data = self.data.copy()
        data['RSI'] = self.calculate_rsi(data['Close'])
        data['OBV'] = self.calculate_obv(data['Close'], data['Volume'])
        data['DPO'] = self.calculate_dpo(data['Close'])
        return data

    def calculate_rsi(self, prices, n=14):
        delta = prices.diff(1)
        up, down = delta.copy(), delta.copy()
        up[up < 0] = 0
        down[down > 0] = 0
        roll_up = up.rolling(n).mean()
        roll_down = down.rolling(n).mean().abs()
        RS = roll_up / roll_down
        RSI = 100.0 - (100.0 / (1.0 + RS))
        return RSI

    def calculate_obv(self, prices, volume):
        obv = volume.copy()
        obv[prices < obv.shift(1)] = -obv[prices < obv.shift(1)]
        obv[prices > obv.shift(1)] = obv[prices > obv.shift(1)]
        return obv.cumsum()

    def calculate_dpo(self, prices):
        dpo = prices.copy()
        dpo = dpo.rolling(30).mean()
        dpo = dpo.rolling(10).mean()
        return dpo

    def prepare_data(self):
        data = self.calculate_indicators()
        data['Date'] = data.index
        data['Open'] = data['Open'].shift(-3)
        data['Low'] = data['Low'].shift(-3)
        data['Close'] = data['Close'].shift(-3)
        data = data.dropna()
        data = data[['Date', 'Open', 'Low', 'Close', 'RSI', 'OBV', 'DPO']]
        return data

    def scale_data(self, data):
        self.scaler = MinMaxScaler()
        data[['Open', 'Low', 'Close', 'RSI', 'OBV', 'DPO']] = self.scaler.fit_transform(data[['Open', 'Low', 'Close', 'RSI', 'OBV', 'DPO']])
        return data

    # def create_model(self, units, dropout, learning_rate):
    #     model = Sequential()
    #     model.add(LSTM(units=units, return_sequences=True, input_shape=(self.data.shape[1], 1)))
    #     model.add(Dense(3))
    #     model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=learning_rate))
    #     return model

    def create_model(self, units, dropout, learning_rate):
        model = Sequential()
        model.add(LSTM(units=units, return_sequences=False, input_shape=(self.data.shape[1], 1)))
        model.add(Dense(3))
        model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=learning_rate))
        return model

    def train_model(self, data, units, dropout, learning_rate):
        X = data.drop(['Date', 'Open', 'Low', 'Close'], axis=1)
        y = data[['Open', 'Low', 'Close']]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        model = self.create_model(units, dropout, learning_rate)
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001)
        model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])
        return model

    def make_predictions(self, model, data):
        predictions = model.predict(data.drop(['Date', 'Open', 'Low', 'Close'], axis=1))
        # predictions = self.scaler.inverse_transform(predictions)
        return predictions

    # def plot_results(self, data, predictions):
    #     fig, ax = mpf.figure(figsize=(12, 6), style='yahoo', )
    #     ax.plot(data['Date'], data['Close'], label='Actual Close')
    #     ax.plot(data['Date'], predictions[:, 2], label='Predicted Close')
    #     ax.plot(data['Date'], data['Open'], label='Actual Open')
    #     ax.plot(data['Date'], predictions[:, 0], label='Predicted Open')
    #     ax.plot(data['Date'], data['Low'], label='Actual Low')
    #     ax.plot(data['Date'], predictions[:, 1], label='Predicted Low')
    #     ax.legend()
    #     mpf.show()

    def plot_results(self, data, predictions):
        # Copy st.orig_data dataframe columns open high low close and volume to data
        data = self.orig_data.copy()
        
        addplot = [mpf.make_addplot(data['Close'], color='b', linestyle='-', title='Actual Close'),
                   mpf.make_addplot(predictions[:, 2], color='b', linestyle='--', title='Predicted Close'),
                   mpf.make_addplot(data['Open'], color='g', linestyle='-', title='Actual Open'),
                   mpf.make_addplot(predictions[:, 0], color='g', linestyle='--', title='Predicted Open'),
                   mpf.make_addplot(data['Low'], color='r', linestyle='-', title='Actual Low'),
                   mpf.make_addplot(predictions[:, 1], color='r', linestyle='--', title='Predicted Low')]
        mpf.plot(data, type='line', addplot=addplot, volume=False, style='yahoo')
        mpf.show()

    def bayesian_optimization(self):
        def optimize_model(units, dropout, learning_rate):
            model = self.create_model(int(units), dropout, learning_rate)
            X = self.data.drop(['Date', 'Open', 'Low', 'Close'], axis=1)
            y = self.data[['Open', 'Low', 'Close']]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            early_stopping = EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001)
            model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])
            loss = model.evaluate(X_test, y_test)
            return -loss

        optimizer = BayesianOptimization(f=optimize_model, pbounds={'units': (10, 100), 'dropout': (0.1, 0.5), 'learning_rate': (0.001, 0.1)}, random_state=42)
        optimizer.maximize(init_points=10, n_iter=20)
        print(optimizer.max)

    def run(self):
        data = self.prepare_data()
        data_scaled = self.scale_data(data)
        self.bayesian_optimization()
        model = self.train_model(data_scaled, units=50, dropout=0.2, learning_rate=0.001)
        # Modify pred_scaled suche that it includes the Date column from the data_scaled dataframe
        # pred_scaled = data_scaled.drop(['Open', 'Low', 'Close'], axis=1)
        pred_scaled = self.make_predictions(model, data)

        self.plot_results(data, predictions)

predictor = StockPredictor('AAPL', '2020-01-01', '2022-02-26')
predictor.run()


[*********************100%***********************]  1 of 1 completed

|   iter    |  target   |  dropout  | learni... |   units   |
-------------------------------------------------------------
Epoch 1/50



/home/dsidlo/anaconda3/envs/pricepredict/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 13641.5586 - val_loss: 8007.8105
Epoch 2/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6888.0288 - val_loss: 3462.7881
Epoch 3/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2734.2766 - val_loss: 1643.8048
Epoch 4/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1351.6381 - val_loss: 1238.1309
Epoch 5/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 982.1923 - val_loss: 1240.7466
Epoch 6/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 879.3528 - val_loss: 1262.7638
Epoch 7/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 904.8337 - val_loss: 1251.6245
Epoch 8/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 938.8983 - val_loss: 1244.5879
Epoch 9/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 895.2655 - val_loss: 1239.2878
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1280.6838 
| 1         | -1.239e+0 | 0.2498    | 0.09512   | 75.88     |
Epoch 1/50


/home/dsidlo/anaconda3/envs/pricepredict/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

/home/dsidlo/anaconda3/envs/pricepredict/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 15503.7012 - val_loss: 14486.8887
Epoch 2/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 14869.2471 - val_loss: 13401.6445
Epoch 3/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 13546.4668 - val_loss: 12173.7246
Epoch 4/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 12085.5068 - val_loss: 11041.4678
Epoch 5/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 11116.3994 - val_loss: 10008.4150
Epoch 6/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 9947.9824 - val_loss: 9081.0225
Epoch 7/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 9323.8477 - val_loss: 8234.3145
Epoch 8/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8466.1270 - val_loss: 7465.9482
Epoch 9/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 7563.9146 - val_loss: 6773.6128
Epoch 10/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6720.9678 - val_loss: 6147.5767
Epoch 11/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6273.5347 - val_loss: 5571.4551
Epoch 12

/home/dsidlo/anaconda3/envs/pricepredict/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.3024 - val_loss: 0.2235
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2062 - val_loss: 0.1344
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1162 - val_loss: 0.0803
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0678 - val_loss: 0.0818
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0625 - val_loss: 0.0747
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0580 - val_loss: 0.0718
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0604 - val_loss: 0.0702
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0567 - val_loss: 0.0687
Epoch 9/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0564 - val_loss: 0.0664
Epoch 10/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0555 - val_loss: 0.0650
Epoch 11/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0502 - val_loss: 0.0632
Epoch 12/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0507 - val_loss: 0.0606

NameError: name 'predictions' is not defined